In [1]:
import random
from lang import load_data_int_seq
from utils import eq_encoder, is_eq_valid,normalize_0_1
from typing import List
from models.rl.env import IntegerSequenceEnv, get_current_position, encode_with_lang, decode_with_lang, eq_to_seq
import gym, ray
from ray.rllib.agents import ppo
from ray.rllib.agents import dqn
import numpy as np

%load_ext autoreload
%autoreload 2

MAX_PENALTY_MAGNITUDE = 999.0

In [2]:
ray.init(ignore_reinit_error=True)


2021-07-08 11:38:48,685	INFO services.py:1272 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.2.103',
 'raylet_ip_address': '192.168.2.103',
 'redis_address': '192.168.2.103:6379',
 'object_store_address': '/tmp/ray/session_2021-07-08_11-38-46_800453_15464/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-07-08_11-38-46_800453_15464/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-07-08_11-38-46_800453_15464',
 'metrics_export_port': 60846,
 'node_id': '2a840fe418cb012be2f15b380e5836dcb9ec842da05109495fc59e1d'}

In [15]:
output_lang, input_lang, train, X_test, y_test = load_data_int_seq()
train = train[:100]

In [16]:
def compare_sequences(output_sequence: List[int], target_sequence: List[int]) -> float:

    if len(output_sequence) != len(target_sequence):
        raise AssertionError("sequence size don't match: " + ','.join(str(e) for e in output_sequence)
 + " | " + ','.join(str(e) for e in target_sequence))

    magnitude: float = 0.0

    combined_seq = np.vstack([output_sequence, target_sequence]) 
    norm_comb_seq = normalize_0_1(combined_seq)

    norm_output_seq = norm_comb_seq[0]
    norm_target_seq = norm_comb_seq[1]

    
    for x, y in zip(norm_target_seq, norm_output_seq):
        magnitude += abs(x - y)#**2

    # magnitude /= len(norm_target_seq)

    return 10 - (magnitude * 100)



In [17]:
compare_sequences([1,2, 3, 100], [1,2,3, 100])

10.0

In [18]:
def evaluate_candidate_eq(candidate: str, int_seq: List[int]) -> float:
    if is_eq_valid(candidate) == False:
        return -MAX_PENALTY_MAGNITUDE

    output_sequence = eq_to_seq(candidate, 9)

    if np.count_nonzero(output_sequence) < 1:
        return -MAX_PENALTY_MAGNITUDE

    return compare_sequences(output_sequence, int_seq)

In [19]:
env = IntegerSequenceEnv({"data": train, "output_length": 9, "input_lang": input_lang, "output_lang": output_lang, "evaluate": evaluate_candidate_eq})

In [20]:
env.step(5)

(([7, -1, -1, -1, -1, -1, -1, -1, -1],
  [249, 250, 153, 251, 252, 253, 254, 255, 189]),
 0,
 False,
 {})

In [21]:
env.step(1)

(([7, 3, -1, -1, -1, -1, -1, -1, -1],
  [249, 250, 153, 251, 252, 253, 254, 255, 189]),
 0,
 False,
 {})

In [22]:
env.reset()

([-1, -1, -1, -1, -1, -1, -1, -1, -1], [2, 28, 3, 64, 6, 71, 72, 73, 74])

In [40]:
trainer = ppo.PPOTrainer(env=IntegerSequenceEnv, config={
    "env_config": {
        "data": train,
        "output_length": 9,
        "input_lang": input_lang,
        "output_lang": output_lang,
        "evaluate": evaluate_candidate_eq
    },
    "num_envs_per_worker": 2,
    # "train_batch_size": 10000,
    "num_workers": 12,
    "framework": "torch"
})

2021-07-08 11:55:12,323	WARNING ppo.py:135 -- `train_batch_size` (4000) cannot be achieved with your other settings (num_workers=12 num_envs_per_worker=2 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 166.
2021-07-08 11:55:24,523	INFO trainable.py:101 -- Trainable.setup took 12.220 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2021-07-08 11:55:24,528	WARNING util.py:53 -- Install gputil for GPU system monitoring.


In [41]:

for i in range(100):
   # Perform one iteration of training the policy with PPO
   result = trainer.train()
   print(result["episode_reward_mean"])
#    print(result)

   if i % 100 == 0:
       checkpoint = trainer.save()
       print("checkpoint saved at", checkpoint, "i: ", i)

2021-07-08 11:55:39,128	WARNING worker.py:1123 -- The actor or task with ID ffffffffffffffffda9085864aba0920be81ccaf01000000 cannot be scheduled right now. It requires {CPU: 1.000000} for placement, but this node only has remaining {0.000000/12.000000 CPU, 6.662329 GiB/6.662329 GiB memory, 3.331165 GiB/3.331165 GiB object_store_memory, 1.000000/1.000000 node:192.168.2.103}
. In total there are 0 pending tasks and 8 pending actors on this node. This is likely due to all cluster resources being claimed by actors. To resolve the issue, consider creating fewer actors or increase the resources available to this Ray cluster. You can ignore this message if this Ray cluster is expected to auto-scale or if you specified a runtime_env for this task or actor because it takes time to install.


KeyboardInterrupt: 

In [ ]:
result = trainer.train()

In [ ]:
result["episode_reward_max"]